## **Bibliotecas**

In [17]:
from pathlib import Path
import yaml
import numpy as np
import pandas as pd
import datetime as dt
from dateutil.relativedelta import relativedelta
from simpledbf import Dbf5
from tqdm import tqdm
from collections import defaultdict
from sqlalchemy import create_engine

from utils.sql_utils import perform_query
from data.load_sources import load_sources

## **Configuração e Input**

Única entrada em que é necessário modificar de rotina para rotina.

In [104]:
config_path = Path.cwd().joinpath("configs")
# -- Essa linha muda de rotina para rotina: deve ser incluída o nome do arquivo de configuração usado para o linkage atual.
config_fname = "base_06mai2025.yml"

In [105]:
with open(config_path.joinpath(config_fname), "r") as f:
    config = yaml.safe_load(f)

database_path = Path(config['database']['path'])
database_name = config['database']['name']
output_filename = config['database']['excel_name']

previous_database_path = Path(config['previous_database']['path'])
previous_database_name = config['previous_database']['name']
previous_output_filename = config['previous_database']['excel_name']

## **Selecionar pares de registros e criar tabela final**

In [106]:
engine_url = f"sqlite:///{database_path.joinpath(database_name)}"
engine = create_engine(engine_url)


'''
    Regra usada para definir pares que realmente correspondem à mesma pessoa:
    Probabilidade do primeiro modelo menor que 0.45 E Probabilidade do terceiro modelo menor que 0.45.
    
    Outras regras podem ser implementadas, mas isso acarreta em um número diferente de registros 
    linkados. A regra acima foi testada empiracamente e foi a que retornou maior confiança na qualidade
    dos pares gerados.
'''
q = f'''
    SELECT 
        *
    FROM likely_positive_pairs
    WHERE PROBA_NEGATIVO_MODELO_1 <= 0.45 AND PROBA_NEGATIVO_MODELO_3 <= 0.45 
'''
# -- if needed to download data again, uncomment the lines below
positive_sd_df = perform_query(q, engine=engine)
positive_sd_df["SIVEP_ID"] = positive_sd_df["FMT_ID"].apply(lambda x: x.split("-")[0])
positive_sd_df["SIM_ID"] = positive_sd_df["FMT_ID"].apply(lambda x: x.split("-")[1])
positive_sd_df["Found"] = [ 'S' for n in range(positive_sd_df.shape[0])  ]
print(positive_sd_df["SIVEP_ID"].unique().shape)
pairs = positive_sd_df[["SIVEP_ID", "SIM_ID"]]
positive_sd_df.head(4)

# -- pares adicionados manualmente por não serem capturados de maneira apropriada
extra_pairs = pd.DataFrame({
    "SIM_ID": ["36824355", "36854428"],
    "SIVEP_ID": ["31725037750178", "31734376382319"]
})
pairs = pd.concat([pairs, extra_pairs])

# -- regra manual adicionada
# ---- 1. remover par (31738027258136, 36854428)
remover1 = (pairs["SIVEP_ID"]=="31738027258136") & (pairs["SIM_ID"]=="36854428")
pairs = pairs[~remover1]
print(pairs.shape)

(2296,)
(2320, 2)


In [107]:
# -- carregar os dados originais
sivep_df, sim_df = load_sources(config)

SIVEP-GRIPE:
	No. Registros: 13,743
	Notificação mais antiga: 2023-12-31 00:00:00
	Notificação mais recente: 2025-05-06 00:00:00

	Sintoma mais antigo: 2023-12-31 00:00:00
	Sintoma mais recente: 2025-05-04 00:00:00

SIM:
	No. Registros: 82,438
	Óbito mais antigo: 2024-01-01 00:00:00
	Óbito mais recente: 2025-04-30 00:00:00



### **SIM**

In [108]:
# -- regra SIM: Causa básica B342
covid_sim = sim_df[(pd.notna(sim_df["CAUSABAS"])) & (sim_df["CAUSABAS"]=="B342")]
found_ids = pairs["SIM_ID"].unique()
# -- encontados
found_covid_sim = covid_sim[covid_sim["NUMERODO"].isin(found_ids)].reset_index(drop=True)
# -- não encontrados
nofound_covid_sim = covid_sim[~covid_sim["NUMERODO"].isin(found_ids)].reset_index(drop=True)
print("SIM:")
print(f"Encontrados: {found_covid_sim.shape[0]}\nNão encontrados: {nofound_covid_sim.shape[0]}")

SIM:
Encontrados: 113
Não encontrados: 40


In [109]:
print("Exemplos entre os encontrados:")
cols = ["NUMERODO", "DTOBITO", "CAUSABAS", "CODESTAB", "DTNASC", "CODMUNRES", "BAIRES"]
found_covid_sim[cols].head(5)

Exemplos entre os encontrados:


,NUMERODO,DTOBITO,CAUSABAS,CODESTAB,DTNASC,CODMUNRES,BAIRES
0,32908486,2024-05-15,B342,2372568,1949-11-12,230445,NaN
1,32907536,2024-12-13,B342,2372568,1938-04-29,230445,NaN
2,33715176,2024-01-16,B342,2611309,1948-10-12,230540,POSTO CONTINENTAL
3,35965791,2024-07-20,B342,2372479,1941-06-23,230430,ZONA RURAL
4,34530024,2024-06-24,B342,2563460,2022-08-03,230425,CAVALO BRAVO


In [110]:
print("Exemplos entre os não encontrados:")
cols = ["NUMERODO", "DTOBITO", "CAUSABAS", "CODESTAB", "DTNASC", "CODMUNRES", "BAIRES"]
nofound_covid_sim[cols].head(5)

Exemplos entre os não encontrados:


,NUMERODO,DTOBITO,CAUSABAS,CODESTAB,DTNASC,CODMUNRES,BAIRES
0,35960855,2024-01-07,B342,2561069,1931-01-10,230820,ZONA RURAL
1,36824694,2024-12-21,B342,2561069,1928-02-10,230820,ZONA RURAL
2,36851481,2024-06-09,B342,2611201,1936-06-08,230185,CENTRO
3,32908500,2024-12-15,B342,2561042,1946-12-17,230535,REDONDA
4,35966143,2024-01-01,B342,2561409,1929-08-09,231090,NaN


### **SIVEP**

In [111]:
new_sivep_df2 = sivep_df.copy()
# -- se um registro do sivep é linkado com mais de um registro do sim, aqui somente um vai ser escolhido.
sivep_to_sim = defaultdict(lambda: np.nan, zip(pairs["SIVEP_ID"], pairs["SIM_ID"]))
new_sivep_df2["SIM_ID"] = new_sivep_df2["ID"].map(sivep_to_sim)

# -- registros do SIVEP que foram encontrados no SIM (mesma pessoa)
sivep_found = new_sivep_df2[pd.notna(new_sivep_df2["SIM_ID"])]
# -- registros do SIVEP que não foram encontrados no SIM
sivep_nofound = new_sivep_df2[pd.isna(new_sivep_df2["SIM_ID"])]

cols_sivep = ["NU_NOTIFIC", "NM_PACIENT", "DT_NOTIFIC", "DT_SIN_PRI", "CO_UNI_NOT", "CO_MUN_RES", "EVOLUCAO", "CLASSI_FIN", "CRITERIO", "DT_EVOLUCA", "PCR_SARS2", "RES_AN", "PCR_RESUL"]
cols_sim = ["NUMERODO", "NOME", "CAUSABAS", "LINHAA", "LINHAB", "LINHAC", "LINHAD", "LINHAII", "DTOBITO", "CODMUNRES", "CODESTAB"]

linkage = sivep_found.merge(sim_df[cols_sim], left_on="SIM_ID", right_on="NUMERODO", how="left")
linkage1 = linkage[cols_sivep+cols_sim]

# -- entre os encontrados, aplicar regra: EVOLUCAO '2', CLASSI_FIN '5' E (PCR_SARS2 '1' OU RES_AN '1' OU PCR_RESUL '1')
cond_pcr = (pd.notna(linkage1["PCR_SARS2"])) & (linkage1["PCR_SARS2"]=='1')
cond_res = (pd.notna(linkage1["RES_AN"])) & (linkage1["RES_AN"]=='1')
cond_pcr_resul = (pd.notna(linkage1["PCR_RESUL"])) & (linkage1["PCR_RESUL"]=='1')
cond_evol = (pd.notna(linkage1["EVOLUCAO"])) & (linkage1["EVOLUCAO"]=='2')
cond_classi = (pd.notna(linkage1["CLASSI_FIN"])) & (linkage1["CLASSI_FIN"]=='5')
linkage_sivep_covid = linkage1[((cond_pcr) | (cond_res) | (cond_pcr_resul)) & (cond_evol) & (cond_classi) ]

cols_order = ["NU_NOTIFIC", "NUMERODO"] + cols_sivep[1:] + cols_sim[1:]
linkage_sivep_covid = linkage_sivep_covid[cols_order].copy()
print(f"Entre os que cumpre a regra, total encontrados: {linkage_sivep_covid.shape[0]}")


# -- entre os não encontrados, aplicar regra: EVOLUCAO '2', CLASSI_FIN '5' E (PCR_SARS2 '1' OU RES_AN '1' OU PCR_RESUL '1')
cond_pcr = (pd.notna(sivep_nofound["PCR_SARS2"])) & (sivep_nofound["PCR_SARS2"]=='1')
cond_res = (pd.notna(sivep_nofound["RES_AN"])) & (sivep_nofound["RES_AN"]=='1')
cond_pcr_resul = (pd.notna(sivep_nofound["PCR_RESUL"])) & (sivep_nofound["PCR_RESUL"]=='1')
cond_evol = (pd.notna(sivep_nofound["EVOLUCAO"])) & (sivep_nofound["EVOLUCAO"]=='2')
cond_classi = (pd.notna(sivep_nofound["CLASSI_FIN"])) & (sivep_nofound["CLASSI_FIN"]=='5')
linkage_sivep_covid_no_sim = sivep_nofound[((cond_pcr) | (cond_res) | (cond_pcr_resul)) & (cond_evol) & (cond_classi) ]
print(f"Entre os que cumprem a regra, total não encontrados: {linkage_sivep_covid_no_sim.shape[0]}")

Entre os que cumpre a regra, total encontrados: 138
Entre os que cumprem a regra, total não encontrados: 7


In [112]:
final_df = pd.concat([linkage_sivep_covid, linkage_sivep_covid_no_sim[cols_sivep]])
final_df = pd.concat([final_df, nofound_covid_sim[cols_sim]])

final_df["SIM_E_SIVEP"] = final_df[["NU_NOTIFIC", "NUMERODO"]].apply(lambda x: 'S' if pd.notna(x["NU_NOTIFIC"]) and pd.notna(x["NUMERODO"]) else 'N', axis=1)
final_df["SOMENTE_SIVEP"] = final_df[["NU_NOTIFIC", "NUMERODO"]].apply(lambda x: 'S' if pd.notna(x["NU_NOTIFIC"]) and pd.isna(x["NUMERODO"]) else 'N', axis=1)
final_df["SOMENTE_SIM"] = final_df[["NU_NOTIFIC", "NUMERODO"]].apply(lambda x: 'S' if pd.isna(x["NU_NOTIFIC"]) and pd.notna(x["NUMERODO"]) else 'N', axis=1)

print(f"Tamanho final de registros: {final_df.shape[0]}")
final_df.head(4)

Tamanho final de registros: 185


,NU_NOTIFIC,NUMERODO,NM_PACIENT,DT_NOTIFIC,DT_SIN_PRI,CO_UNI_NOT,CO_MUN_RES,EVOLUCAO,CLASSI_FIN,CRITERIO,...,LINHAB,LINHAC,LINHAD,LINHAII,DTOBITO,CODMUNRES,CODESTAB,SIM_E_SIVEP,SOMENTE_SIVEP,SOMENTE_SIM
2,31704242190717,36815696,ROBERTO GONCALVES MAZZA,2024-01-02,2024-01-01,2806215,230765,2,5,1,...,NaN,NaN,NaN,NaN,2024-01-05,230765,2497654,S,N,N
4,31704279893164,36802995,SANTIAGO GOMES FERREIRA,2024-01-03,2024-01-02,6779522,230730,2,5,1,...,*N188,NaN,NaN,*B342*U071*I10,2024-01-03,230730,6779522,S,N,N
23,31704456019088,36803351,JULIO CESAR PETRONILO DE MOURA,2024-01-05,2024-01-02,6779522,230830,2,5,1,...,*J180,*G809,NaN,*B342*U071,2024-01-12,230830,6779522,S,N,N
36,31704799193216,36813964,JOAQUIM FERREIRA LIMA,2024-01-09,2024-01-08,7692595,231270,2,5,1,...,*B342*U072,NaN,NaN,NaN,2024-01-08,231270,0086673,S,N,N


### **Comparar com o linkage anterior**

In [113]:
final_df1 = final_df.rename({"NM_PACIENT": "NOME (SIVEP)", "NOME": "NOME (SIM)"}, axis=1)
#final_df2 = final_df1.copy()#[(final_df1["DT_NOTIFIC"]>=dt.datetime(2024, 1, 1)) | (pd.isna(final_df1["DT_NOTIFIC"]))]
#final_df2.to_excel(workpath.joinpath(config['output_filename']))

In [114]:
previous_df = pd.read_excel(previous_database_path.joinpath(previous_output_filename), dtype={"NU_NOTIFIC": str, "NUMERODO": str}, index_col=0)
previous_df["KEY"] = previous_df["NU_NOTIFIC"].astype(str) + previous_df["NUMERODO"].astype(str)

final_test = final_df1.copy()
final_test["KEY"] = final_test["NU_NOTIFIC"].astype(str) + final_test["NUMERODO"].astype(str)
previous_changed_df = previous_df[~previous_df["KEY"].isin(final_test["KEY"])]#[["NU_NOTIFIC", "NUMERODO", "NOME (SIVEP)", "DT_NOTIFIC", "DT_SIN_PRI", "CO_MUN_RES", "EVOLUCAO", "CLASSI_FIN"]]
previous_changed_df = previous_changed_df[~previous_changed_df["NUMERODO"].isin(final_test["NUMERODO"])]
#previous_changed_df = previous_changed_df[~previous_changed_df["NUMERODO"].isin(final_test["NUMERODO"])]

cols = ["NU_NOTIFIC", "NUMERODO", "DT_NOTIFIC", "DT_SIN_PRI", "EVOLUCAO", "CLASSI_FIN", "PCR_SARS2", "RES_AN", "PCR_RESUL", "CAUSABAS"]
previous_changed_df[cols]

,NU_NOTIFIC,NUMERODO,DT_NOTIFIC,DT_SIN_PRI,EVOLUCAO,CLASSI_FIN,PCR_SARS2,RES_AN,PCR_RESUL,CAUSABAS


#### **Investigação**

Se houver registros no linkage anterior que não estão mais presentes no linkage atual, é necessária uma investigação das razões. 
Abaixo estão algumas possibilidades.

**Um dos registros não existem mais no banco atualizado**

In [115]:
for index, row in previous_changed_df.iterrows():
    sivep_id, sim_id = row['NU_NOTIFIC'], row["NUMERODO"]

    if sivep_id not in sivep_df["NU_NOTIFIC"].tolist():
        print(f"SIVEP: ID {sivep_id} não está presente na base atualizada")
    else:
        print(f"SIVEP: ID {sivep_id} está presente na base atualizada")

    if sim_id not in sim_df["NUMERODO"].tolist():
        print(f"SIM: ID {sim_id} não está presente na base atualizada")
    else:
        print(f"SIM: ID {sim_id} está presente na base atualizada")
    print("")

**Qual é a causa básica dos óbitos não presentes na base nova e na base antiga?**

In [116]:
for index, row in previous_changed_df.iterrows():
    sim_id, causabas = row["NUMERODO"], row["CAUSABAS"]

    # -- na base nova
    causabas_nova = sim_df[sim_df["NUMERODO"]==sim_id]["CAUSABAS"].iat[0]
    print(f"Óbito {sim_id}: Causa {causabas} (antigo) - Causa {causabas_nova} (nova)")


**Os registros do sivep não mais presentes tiveram regra de seleção modificadas?**

In [117]:
for index, row in previous_changed_df.iterrows():
    sivep_id = row["NU_NOTIFIC"]
    # -- valores antigos
    evol, classi_fin = row["EVOLUCAO"], row["CLASSI_FIN"]
    pcr_sars2, pcr_resul, res_an = row["PCR_SARS2"], row["PCR_RESUL"], row["RES_AN"]

    # -- valores novos
    sel = sivep_df[sivep_df["NU_NOTIFIC"]==sivep_id]
    evol_novo, classi_fin_novo = sel["EVOLUCAO"].iat[0], sel["CLASSI_FIN"].iat[0]
    pcr_sars2_novo, pcr_resul_novo, res_an_novo = sel["PCR_SARS2"].iat[0], sel["PCR_RESUL"].iat[0], sel["RES_AN"].iat[0]

    print(sivep_id)
    print("Valores de regra na base antiga:")
    print(f"EVOL: {evol}, CLASSI_FIN: {classi_fin}")
    print(f"PCR_SARS2: {pcr_sars2}, PCR_RESUL: {pcr_resul}, RES_AN: {res_an}")

    print("Valores de regra na base atualizada:")
    print(f"EVOL: {evol_novo}, CLASSI_FIN: {classi_fin_novo}")
    print(f"PCR_SARS2: {pcr_sars2_novo}, PCR_RESUL: {pcr_resul_novo}, RES_AN: {res_an_novo}")
    print("")


**Qual é a diferença no nome dos indivíduos?**

In [118]:
for index, row in previous_changed_df.iterrows():
    sivep_id, sim_id = row['NU_NOTIFIC'], row["NUMERODO"]

    # -- nomes no antigo:
    print(sivep_id, sim_id)
    nome_sivep, nome_sim = row["NOME (SIVEP)"], row["NOME (SIM)"]
    print("Nomes na base antiga:")
    print(f"SIVEP: {nome_sivep}")
    print(f"SIM: {nome_sim}")

    # -- nomes no antigo:
    sel_sivep = sivep_df[sivep_df["NU_NOTIFIC"]==sivep_id]["NM_PACIENT"].iat[0]
    sel_sim = sim_df[sim_df["NUMERODO"]==sim_id]["NOME"].iat[0]
    print("Nomes na base atualizada:")
    print(f"SIVEP: {sel_sivep}")
    print(f"SIM: {sel_sim}")

**É possível que um registro do SIVEP foi linkado com mais de uma D.O.?**

Aqui, é preciso verificar se a D.O. correta foi escolhida. Senão, é necessário fazer a modificação na primeira célula.

In [119]:
for index, row in previous_changed_df.iterrows():
    sivep_id, sim_id = row['NU_NOTIFIC'], row["NUMERODO"]

    obitos_linkados = pairs[pairs["SIVEP_ID"]==sivep_id]
    if obitos_linkados.shape[0]>1:
        print(f"Mais de um óbito linkado com o registro {sivep_id}:")
        print(f"{row['NOME (SIVEP)']}")
        for obito in obitos_linkados["SIM_ID"].tolist():
            atual = sim_df[sim_df["NUMERODO"]==obito]
            print(f"\tD.O. {atual['NUMERODO'].iat[0]}: {atual['NOME'].iat[0]}, {atual['DTNASC'].iat[0]}")

## **Salvar arquivo final**

In [120]:
with pd.ExcelWriter(database_path.joinpath(output_filename)) as writer:
    final_df1.reset_index(drop=True).to_excel(writer, sheet_name="CRUZAMENTO_ATUALIZADO")  
    previous_changed_df.reset_index(drop=True).to_excel(writer, sheet_name="ANTIGOS_NAO_SATISFAZEM_MAIS") 